In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, Markdown
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score

In [2]:
# Tus datos reales
datos = {
    512:   ([1.000535, 0.530576, 0.282809, 0.165258, 0.108383, 0.077999],
            [1.000459, 0.527290, 0.243084, 0.121492, 0.076564, 0.036679],
            [0.000016, 0.003229, 0.044699, 0.043689, 0.031819, 0.040011]),
    1024:  ([3.889512, 2.424211, 1.049989, 0.598888, 0.379166, 0.268707],
            [3.889434, 2.186563, 1.032492, 0.508392, 0.264917, 0.146860],
            [0.000033, 0.237648, 0.017511, 0.088783, 0.114379, 0.120324]),
    2048:  ([15.486683, 8.015858, 4.050107, 2.105107, 1.307373, 0.887010],
            [15.486493, 7.951011, 2.038692, 1.140401, 0.596007, 0.596007],
            [0.000049, 0.064827, 0.113255, 0.194064, 0.711366, 0.058687]),
    4096:  ([61.914910, 32.019149, 16.124259, 8.415599, 4.817559, 3.376290],
            [61.917420, 31.972901, 16.306005, 8.585585, 4.793292, 2.848708],
            [0.000222, 0.025483, 0.022449, 0.045205, 0.025310, 0.470502]),
    8192:  ([245.923765, 127.623835, 63.993006, 35.688531, 18.576346, 13.119167],
            [245.923176, 127.662219, 63.992255, 33.630006, 17.532977, 13.035616],
            [0.000222, 0.007479, 0.063579, 0.153197, 0.043205, 0.083156])
}

P = np.array([1, 2, 4, 8, 16, 32])

# Modelos a ajustar
def pot_model(P, a, b):
    return a * P**b

def amdahl_model(P, T1, f):
    return T1 * ((1 - f) + f / P)

# Función principal del dashboard
def plot_dashboard(N):
    T_total, T_comp, T_comm = datos[N]
    T_total = np.array(T_total)
    T_comp = np.array(T_comp)
    T_comm = np.array(T_comm)

    # Speedup y eficiencia
    S = T_total[0] / T_total
    E = S / P
    perc_comm = T_comm / T_total * 100

    # Ajuste modelo potencia
    popt_pot, _ = curve_fit(pot_model, P, T_total, p0=[T_total[0], -1])
    T_pot_fit = pot_model(P, *popt_pot)
    r2_pot = r2_score(T_total, T_pot_fit)

    # Ajuste modelo Amdahl
    popt_amdahl, _ = curve_fit(amdahl_model, P, T_total, p0=[T_total[0], 0.95], bounds=([0, 0],[np.inf, 1.0]))
    T_amdahl_fit = amdahl_model(P, *popt_amdahl)
    r2_amdahl = r2_score(T_total, T_amdahl_fit)

    # Subplots
    fig, axs = plt.subplots(3, 2, figsize=(14, 10))

    # Tiempo total + Ajustes
    axs[0,0].plot(P, T_total, 'o-', label='Datos medidos')
    axs[0,0].plot(P, T_pot_fit, '--', label=f'Potencia (b={popt_pot[1]:.3f}, R²={r2_pot:.4f})')
    axs[0,0].plot(P, T_amdahl_fit, ':', label=f'Amdahl (f={popt_amdahl[1]:.3f}, R²={r2_amdahl:.4f})')
    axs[0,0].plot(P, T_total[0]/P, '-', color='gray', label='Curva ideal')
    axs[0,0].set_xscale('log', base=2)
    axs[0,0].set_yscale('log')
    axs[0,0].set_title(f"Tiempo Total y Ajustes (N={N})")
    axs[0,0].set_xlabel("N° Procesos")
    axs[0,0].set_ylabel("Tiempo (s)")
    axs[0,0].legend()
    axs[0,0].grid()

    # Speedup
    axs[0,1].plot(P, S, 'o-', label='Speedup real')
    axs[0,1].plot(P, P, '--', label='Speedup ideal')
    axs[0,1].set_xscale('log', base=2)
    axs[0,1].set_title(f"Speedup (N={N})")
    axs[0,1].set_xlabel("N° Procesos")
    axs[0,1].set_ylabel("Speedup")
    axs[0,1].legend()
    axs[0,1].grid()

    # Eficiencia
    axs[1,0].plot(P, E*100, 'o-', color='green', label='Eficiencia (%)')
    axs[1,0].set_xscale('log', base=2)
    axs[1,0].set_title(f"Eficiencia (N={N})")
    axs[1,0].set_xlabel("N° Procesos")
    axs[1,0].set_ylabel("Eficiencia (%)")
    axs[1,0].legend()
    axs[1,0].grid()

    # Comunicación absoluta
    axs[1,1].bar(P, T_comp, label='Cómputo')
    axs[1,1].bar(P, T_comm, bottom=T_comp, label='Comunicación')
    axs[1,1].set_xscale('log', base=2)
    axs[1,1].set_title(f"Tiempos Cómputo vs Comunicación (N={N})")
    axs[1,1].set_xlabel("N° Procesos")
    axs[1,1].set_ylabel("Tiempo (s)")
    axs[1,1].legend()

    # % Comunicación
    axs[2,0].plot(P, perc_comm, 'o-', color='red', label='% Comunicación')
    axs[2,0].set_xscale('log', base=2)
    axs[2,0].set_title(f"Porcentaje de Comunicación (N={N})")
    axs[2,0].set_xlabel("N° Procesos")
    axs[2,0].set_ylabel("% de Comunicación")
    axs[2,0].legend()
    axs[2,0].grid()

    # Texto resumen
    axs[2,1].axis('off')
    resumen = f"""
    Para N={N}:
    - Speedup máximo: {S[-1]:.2f}x en {P[-1]} procesos.
    - Eficiencia final: {E[-1]*100:.2f}%.
    - % comunicación en 32 procesos: {perc_comm[-1]:.2f}%.
    - Modelo Potencia: b = {popt_pot[1]:.3f}, R² = {r2_pot:.4f}
    - Modelo Amdahl: f = {popt_amdahl[1]:.3f}, R² = {r2_amdahl:.4f}
    """
    axs[2,1].text(0.01, 0.5, resumen, fontsize=12, verticalalignment='center')

    plt.tight_layout()
    plt.show()

    # Conclusiones dinámicas
    display(Markdown(f"""
    **Conclusiones para N={N}:**

    - El modelo de Amdahl indica que la fracción paralelizable es de aproximadamente {popt_amdahl[1]*100:.2f}%.
    - El exponente del modelo potencia es {popt_pot[1]:.3f}, lo que indica {"excelente" if popt_pot[1] < -0.95 else "moderada"} escalabilidad.
    - El R² para Amdahl es {r2_amdahl:.4f}, demostrando que {"ajusta muy bien a los datos." if r2_amdahl > 0.98 else "no ajusta perfectamente a los datos."}
    - En 32 procesos, la eficiencia es de {E[-1]*100:.2f}%.
    - {"✅ El escalado es excelente." if E[-1] > 0.7 else "⚠️ El escalado está limitado por fracción secuencial o comunicación."}
    """))

# Dropdown para Voila
dropdown = widgets.Dropdown(
    options=list(datos.keys()),
    value=512,
    description='Malla N:',
)

widgets.interact(plot_dashboard, N=dropdown);


interactive(children=(Dropdown(description='Malla N:', options=(512, 1024, 2048, 4096, 8192), value=512), Outp…